### 0. Import libraries and implement functions

In [2]:
from pylatexenc.latexwalker import LatexWalker, LatexEnvironmentNode, LatexCharsNode, LatexCommentNode,\
                                    LatexGroupNode, LatexMathNode, LatexMacroNode, LatexSpecialsNode
import sys, re, os

In [3]:
def find_tex_files(dir, max_depth):
    if max_depth == 0 or not os.path.isdir(dir):
        return []

    fs = os.listdir(dir)

    if dir != ".":
        fs = [dir + "/" + f for f in fs]

    sub = [f for f in fs if os.path.isdir(f)]
    fs = [f for f in fs if os.path.isfile(f) and f.endswith(".tex")]

    for d in sub:
        fs.extend(find_tex_files(d, max_depth - 1))
    return fs

In [4]:
def find_begin_document(files):
    for file in files:
        with open(file, "r") as f:
            text = f.read()
            if r"\begin{document}" in text:
                return file
    return None

In [5]:
def find_include(tex):
    pattern = r"\\(?:include|input)\{([\w\d\/_]+)\}"
    lines = tex.split("\n")
    files = []
    for i in range(len(lines)):
        line = lines[i]

        percent_idx = 0
        while True:
            percent_idx = line.find("%", percent_idx)
            if percent_idx == -1:
                break

            if percent_idx == 0 or (percent_idx > 0 and line[percent_idx - 1] != '\\'):
                break
            percent_idx += 1

        if percent_idx == 0:
            continue
        elif percent_idx != -1:
            line = line[:percent_idx - 1]

        captured = re.findall(pattern, line)
        if len(captured) > 0:
            files.extend(captured)

    return list(set(files))

In [6]:
def get_latex_nodes(fp):
    with open(fp, "r") as f:
        text = f.read()

    newcommand_pattern = r"^\\newcommand\{([^}]+)\}(?:\[[^]]+\])?\{(.+)\}$"
    newcommands = re.findall(newcommand_pattern, text, flags=re.M)
    # print(newcommands)
    for (short_hand, cmd) in newcommands:
        short_hand = short_hand.replace("\\", "\\\\")
        cmd = cmd.replace("\\", "\\\\")
        text = re.sub(short_hand, cmd, text)

    text = re.sub(newcommand_pattern, "", text)

    bibitem_pattern = r"^\\bibitem{([^}]+)}\s+(.+)$"
    text = re.sub(bibitem_pattern, "", text)

    w = LatexWalker(text)
    nodes, _, _ = w.get_latex_nodes()
    return nodes

In [7]:
SENTENCE_PATTERN = re.compile(r"(?<!\b[A-Z])(?<![Ee][Tt] [Aa][Ll])\.\s+(?=[A-Z])")

def split_sentences(text, level):
    sentences = SENTENCE_PATTERN.split(text)
    sentences = [(sentence.strip(), level) for sentence in sentences]

    return sentences

In [8]:
LEVELS = {
    "document": 0,
    "abstract": 1,
    "section": 1,
    "subsection": 2,
    "subsubsection": 3,
    "paragraph": 4,
    "subparagraph": 5,
    "itemize": 6,
    "item": 7,
    "leaf": 8,
}

In [44]:
def hierarchy_nodes(nodes, working_dir, append_trailing=False):
    text = ""
    tokens = []
    for node in nodes:
        if node == None or isinstance(node, LatexCommentNode):
            continue

        if isinstance(node, LatexCharsNode):
            chars = node.chars
            text += chars
        elif isinstance(node, LatexMathNode):
            text += node.latex_verbatim()
        elif isinstance(node, LatexGroupNode):
            text += node.latex_verbatim()
            # tokens.extend(node.nodelist)
        elif isinstance(node, LatexEnvironmentNode):
            env_name = node.environmentname.lower()
            if env_name in ["figure", "figure*", "equation", "equation*", "align", "align*", "table", "remark", "remark*"]:
                tokens.extend(split_sentences(text, LEVELS["leaf"]))
                tokens.append((node.latex_verbatim(), LEVELS["leaf"]))
                text = ""
            elif env_name == "itemize":
                tokens.extend(split_sentences(text, LEVELS["leaf"]))
                tokens.append((env_name, LEVELS[env_name]))
                latex = node.latex_verbatim()
                # pattern = r"\\(begin|end)\{" + env_name + r"\}" + r"(\[[^]]+\])?"
                pattern = r"\\(begin|end)\{itemize\}(\[[^]]+\])?"
                latex = re.sub(pattern, "", latex, flags=re.IGNORECASE).strip()

                items = re.split(r"\\item", latex, flags=re.IGNORECASE)
                for item in items:
                    item = item.strip()
                    if len(item) == 0:
                        continue

                    tokens.append(("item", LEVELS["item"]))
                    tokens.extend(split_sentences(item, LEVELS["leaf"]))

            elif env_name in "document":
                tokens.append((env_name, LEVELS[env_name]))
                sub_tokens = hierarchy_nodes(node.nodelist, working_dir, True)
                tokens.extend(sub_tokens)
            elif env_name in "abstract":
                tokens.append((env_name, LEVELS[env_name]))
                latex = node.latex_verbatim()
                latex = re.sub(r"\\(begin|end)\{abstract\}", "", latex, flags=re.IGNORECASE).strip()
                tokens.extend(split_sentences(latex, LEVELS["leaf"]))
            else:
                # print(env_name, "[Environment]")
                sub_tokens = hierarchy_nodes(node.nodelist, working_dir, True)
                tokens.extend(sub_tokens)
        elif isinstance(node, LatexMacroNode):
            if node.macroname in ["input", "include"]:
                if len(node.nodeargd.argnlist) != 1 or len(node.nodeargd.argnlist[0].nodelist) != 1 \
                        or not isinstance(node.nodeargd.argnlist[0].nodelist[0], LatexCharsNode):
                    print("\t"*tabs, "?Empty?", f"[\\{node.macroname}]")
                    exit(1)
                else:
                    tokens.extend(split_sentences(text, LEVELS["leaf"]))
                    text = ""

                    fp = os.path.join(working_dir, node.nodeargd.argnlist[0].nodelist[0].chars)
                    dependencies_nodes = get_latex_nodes(fp)
                    print("Parse", fp)
                    sub_tokens = hierarchy_nodes(dependencies_nodes, working_dir, True)
                    tokens.extend(sub_tokens)

                    text = ""
            elif node.macroname in ["section", "subsection", "subsubsection", "paragraph", "subparagraph"]:
                tokens.extend(split_sentences(text, LEVELS["leaf"]))
                latex = node.latex_verbatim()
                latex = re.sub(r"\\" + node.macroname + r"\{", "", latex, flags=re.IGNORECASE).strip()
                tokens.append((latex[:-1], LEVELS[node.macroname]))
                text = ""
            elif node.macroname in ["label", "footnote"]:
                latex = node.latex_verbatim()
                pattern = fr"\\{node.macroname}" + "{[^}]+}"
                latex = re.sub(pattern, "", latex)
                text += latex
            elif node.macroname in ["cite", "citep", "citet"]:
                pass
            else:
                text += node.latex_verbatim()
        elif isinstance(node, LatexSpecialsNode):
            text += node.specials_chars

    if append_trailing and text != "":
        tokens.extend(split_sentences(text, LEVELS["leaf"]))

    return tokens

In [45]:
def add_node(root, stack, elements):
    key = elements[stack[0]][0]
    if len(stack) == 1:
        root[key] = {}
    else:
        if key in root:
            add_node(root[key], stack[1:], elements)

In [46]:
def find_version_elements(version_directory):
    main_tex_fp = os.path.join(version_directory, "main.tex")

    if not os.path.exists(main_tex_fp):
        depth1_tex_files = find_tex_files(version_directory, 1)
        main_tex_fp = find_begin_document(depth1_tex_files)
        if main_tex_fp == None:
            print("Not found main tex file")
            return None

    nodes = get_latex_nodes(main_tex_fp)
    nodes = hierarchy_nodes(nodes, version_directory)
    nodes = [node for node in nodes if len(node[0]) > 0]

    ignore_idx = 0
    while ignore_idx < len(nodes) and nodes[ignore_idx] != ("document", 0):
        ignore_idx += 1
    elements = nodes[ignore_idx:]

    return elements

def hierarchy_elements(elements):
    root = {}
    node_stack = []
    
    for i in range(len(elements)):
        if len(node_stack) > 0:
            last_node_level = elements[node_stack[-1]][1]
        else:
            last_node_level = 0

        current_node_level = elements[i][1]

        if current_node_level > last_node_level:
            node_stack.append(i)
        elif current_node_level == last_node_level:
            if len(node_stack) > 0:
                node_stack[-1] = i
            else:
                node_stack.append(i)
        else:
            while len(node_stack) > 0 and current_node_level <= elements[node_stack[-1]][1]:
                node_stack.pop()

            node_stack.append(i)
        add_node(root, node_stack, elements)

    return root

### 1. Testing

In [64]:
def replace_with_id(root, new_root, ids):
    for key, value in root.items():
        # print(key)
        new_key = ids[key]
        if value != {}:
            new_root[new_key] = {}
            replace_with_id(value, new_root[new_key], ids)
        else:
            new_root[new_key] = value
        

In [70]:
import json
# nodes, node_hierarchy, refs = hierarchy_version("../../23127247_milestone1/2210.16424/tex/2210.16424v1/")
prefix_dir = "../../../23127247_milestone1/"
paper_id = "2210.16578"

paper_dir = os.path.join(prefix_dir, paper_id, "tex")
version = 1

all_version_elements = []
paper_elements = set()
while True:
    version_dir = os.path.join(paper_dir, paper_id + "v" + str(version))
    if not os.path.exists(version_dir):
        break
    version_elements = find_version_elements(version_dir)
    all_version_elements.append(version_elements)
    set_version_elements = [ele[0] for ele in version_elements]
    paper_elements |= set(set_version_elements)
    version += 1

paper_elements = list(paper_elements)
paper_elements_dict = {paper_elements[i]: i for i in range(len(paper_elements))}
# print(json.dumps(paper_elements_dict, indent=4))

hierarchy = {}
for i in range(len(all_version_elements)):
    version_elements = all_version_elements[i]
    root = hierarchy_elements(version_elements)
    new_root = {}
    replace_with_id(root, new_root, paper_elements_dict)
    hierarchy[str(i + 1)] = new_root
print(json.dumps(hierarchy, indent=4))

elements = {paper_id + "_" + str(value): key for key, value in paper_elements_dict.items()}

for key, value in elements.items():
    print(key, value)

{
    "1": {
        "170": {
            "115": {
                "145": {},
                "525": {},
                "528": {},
                "43": {},
                "114": {},
                "305": {},
                "160": {},
                "167": {},
                "199": {}
            },
            "309": {
                "364": {},
                "263": {},
                "479": {},
                "249": {},
                "255": {},
                "38": {},
                "552": {},
                "575": {},
                "536": {},
                "344": {},
                "51": {},
                "368": {},
                "459": {},
                "10": {},
                "353": {},
                "549": {},
                "416": {},
                "501": {},
                "227": {},
                "219": {},
                "358": {},
                "23": {},
                "389": {},
                "483": {},
                "555": {},
 